<a href="https://colab.research.google.com/github/RenzoPL23/Eternity-game/blob/master/CSP_Eternity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

     |████████████████████████████████| 28.9MB 151kB/s 
     |████████████████████████████████| 1.3MB 47.1MB/s 
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


In [ ]:
from ortools.sat.python import cp_model

In [ ]:
# 0 1 2 3 4 5 6 7 8
# - G R V Y A B O R
strFichas="- G R - V V Y V A V V Y Y A V Y V Y Y V G - - G - B V G Y Y A V A V Y V A V V A B - G Y R V O - Y Y V Y V Y V Y V V Y Y Y G - R - O Y B Y Y A Y Y V A V O - B V O Y R - V Y V Y V Y V V Y R - B - O Y O Y V Y Y R - O V R Y B - V V Y Y V B - O - B Y O R - R V B Y G - Y O - G - - R B G G - -"
strFichas2 = strFichas.replace(" ","")
fichas2=[]
for i in range(36):
  ficha=0
  for j in range(4):
    posicion= 10**(3-j)
    if(strFichas2[i*4+j]=="-"):
      ficha+=posicion*0
    elif(strFichas2[i*4+j]=="G"):
      ficha+=posicion*1
    elif(strFichas2[i*4+j]=="R"):
      ficha+=posicion*2
    elif(strFichas2[i*4+j]=="V"):
      ficha+=posicion*3
    elif(strFichas2[i*4+j]=="Y"):
      ficha+=posicion*4
    elif(strFichas2[i*4+j]=="A"):
      ficha+=posicion*5
    elif(strFichas2[i*4+j]=="B"):
      ficha+=posicion*6
    elif(strFichas2[i*4+j]=="O"):
      ficha+=posicion*7
    elif(strFichas2[i*4+j]=="R"):
      ficha+=posicion*8
  fichas2+=[ficha]
fichas2.sort()


In [ ]:
fichas2

[26,
 120,
 631,
 647,
 746,
 747,
 1001,
 1100,
 2023,
 2073,
 2370,
 2460,
 3343,
 3344,
 3344,
 3433,
 3434,
 3434,
 3443,
 3607,
 4102,
 4206,
 4344,
 4353,
 4434,
 4453,
 4454,
 4534,
 4701,
 5334,
 5335,
 5343,
 6014,
 6410,
 7063,
 7420]

In [ ]:
model = cp_model.CpModel()
grid = []
sums=[]
for i in range(6):
    row = []
    sumrow=[]
    for j in range(6):
      four=[]
      for k in range(4):
        four+=[model.NewIntVar(0,9,"x"+str(i)+str(j)+str(k))]
      row+=[four]
      sumrow+=[model.NewIntVarFromDomain(cp_model.Domain.FromValues(fichas2),"sum"+str(i)+str(j))]
    grid +=[row]
    sums+=[sumrow]

sumsAux=[]
for i in range(6):
  for j in range(6):
    sumsAux.append(sums[i][j])
    model.Add(grid[i][j][0]*1000+grid[i][j][1]*100+grid[i][j][2]*10+grid[i][j][3]==sums[i][j])

    if(i!=5):
      model.Add(grid[i][j][2]==grid[i+1][j][0])
    if(j!=5):
      model.Add(grid[i][j][1]==grid[i][j+1][3])
  
countsMatrix=[[[] for i in range(36)]for i in range(36)]
duplicates=[13,14,16,17]

for i in range(6):
  for j in range(6):
    model.Add(grid[0][i][0]==0)
    model.Add(grid[5][i][2]==0)
    model.Add(grid[i][0][3]==0)
    model.Add(grid[i][5][1]==0)

#model.AddDecisionStrategy(sumsAux,cp_model.CHOOSE_FIRST,cp_model.SELECT_MIN_VALUE) #1

#model.AddDecisionStrategy(sumsAux,cp_model.CHOOSE_LOWEST_MIN,cp_model.SELECT_UPPER_HALF) #2
#model.AddDecisionStrategy(sumsAux,cp_model.CHOOSE_HIGHEST_MAX,cp_model.SELECT_LOWER_HALF) #3

for i in range(36):
  for j in range(36):
    countsMatrix[i][j]=model.NewBoolVar("count"+str(i)+str(j))
    model.Add(sumsAux[j]==fichas2[i]).OnlyEnforceIf(countsMatrix[i][j])
    model.Add(sumsAux[j]!=fichas2[i]).OnlyEnforceIf(countsMatrix[i][j].Not())
  if(i in duplicates):
    model.Add(sum(countsMatrix[i]) == 2)
  else:
    model.Add(sum(countsMatrix[i])==1)


# CHOOSE_FIRST, SELECT_MIN_VALUE

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.FEASIBLE:
    for i in range(6):
        for j in range(6):
          for k in range(4):
            print(solver.Value(grid[i][j][k]),end="-")
          print(end=" ") 
        print()
    print(solver.WallTime())
else:
  print("No se puede manito")

0-1-2-0- 0-6-3-1- 0-7-4-6- 0-7-4-7- 0-6-4-7- 0-0-2-6- 
2-3-7-0- 3-3-4-3- 4-4-5-3- 4-4-5-4- 4-3-4-4- 2-0-2-3- 
7-4-2-0- 4-5-3-4- 5-3-3-5- 5-3-4-3- 4-3-5-3- 2-0-7-3- 
2-4-6-0- 3-4-3-4- 3-4-3-4- 4-4-3-4- 5-3-3-4- 7-0-6-3- 
6-4-1-0- 3-3-4-4- 3-4-3-3- 3-3-4-4- 3-4-4-3- 6-0-1-4- 
1-1-0-0- 4-7-0-1- 3-6-0-7- 4-2-0-6- 4-1-0-2- 1-0-0-1- 
0.051905994000000004


# CHOOSE_LOWEST_MIN,SELECT_UPPER_HALF

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.FEASIBLE:
    for i in range(6):
        for j in range(6):
          for k in range(4):
            print(solver.Value(grid[i][j][k]),end="-")
          print(end=" ") 
        print()
    print(solver.WallTime())
else:
  print("No se puede manito")

0-1-2-0- 0-6-3-1- 0-7-4-6- 0-7-4-7- 0-6-4-7- 0-0-2-6- 
2-3-7-0- 3-3-4-3- 4-4-5-3- 4-4-5-4- 4-3-4-4- 2-0-2-3- 
7-4-2-0- 4-5-3-4- 5-3-3-5- 5-3-4-3- 4-3-5-3- 2-0-7-3- 
2-4-6-0- 3-4-3-4- 3-4-3-4- 4-4-3-4- 5-3-3-4- 7-0-6-3- 
6-4-1-0- 3-3-4-4- 3-4-3-3- 3-3-4-4- 3-4-4-3- 6-0-1-4- 
1-1-0-0- 4-7-0-1- 3-6-0-7- 4-2-0-6- 4-1-0-2- 1-0-0-1- 
0.048141453


#CHOOSE_HIGHEST_MAX, SELECT_LOWER_HALF

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.FEASIBLE:
    for i in range(6):
        for j in range(6):
          for k in range(4):
            print(solver.Value(grid[i][j][k]),end="-")
          print(end=" ") 
        print()
    print(solver.WallTime())
else:
  print("No se puede manito")

0-1-2-0- 0-6-3-1- 0-7-4-6- 0-7-4-7- 0-6-4-7- 0-0-2-6- 
2-3-7-0- 3-3-4-3- 4-4-5-3- 4-4-5-4- 4-3-4-4- 2-0-2-3- 
7-4-2-0- 4-5-3-4- 5-3-3-5- 5-3-4-3- 4-3-5-3- 2-0-7-3- 
2-4-6-0- 3-4-3-4- 3-4-3-4- 4-4-3-4- 5-3-3-4- 7-0-6-3- 
6-4-1-0- 3-3-4-4- 3-4-3-3- 3-3-4-4- 3-4-4-3- 6-0-1-4- 
1-1-0-0- 4-7-0-1- 3-6-0-7- 4-2-0-6- 4-1-0-2- 1-0-0-1- 
0.051922997000000005
